In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# https://udemy.com/recommender-systems
# https://deeplearningcourses.com/recommender-systems
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future
import os
import pickle
import numpy as np
import pandas as pd
import numba # Tăng tốc tính toán df
import matplotlib.pyplot as plt
from sklearn.utils import shuffle, class_weight
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten, Concatenate, Dense, Activation, Dropout, BatchNormalization, Lambda, Reshape
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import StratifiedKFold

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

# Train data

In [ ]:
# dataset_2022_clear
path = '/content/drive/MyDrive/Recommendation/'
data_train = pd.read_csv(path + "dataset_2022_clear.csv")
data_train

,new_customer_code,new_sku,purchase_dummy
0,0,0,1
1,1,1,1
2,1,2,1
3,2,3,1
4,2,4,1
...,...,...,...
2790263,26488,2109,1
2790264,26488,4042,1
2790265,26488,6457,1
2790266,373,4025,1


# Test data

In [ ]:
# dataset_test_map
path = '/content/drive/MyDrive/Recommendation/'
data_test = pd.read_csv(path + "dataset_test_map.csv")
data_test

,new_customer_code,new_sku,purchase_dummy
0,23915,1167,1
1,4339,1167,1
2,12775,1167,1
3,6257,1167,1
4,8089,1167,1
...,...,...,...
450305,14900,10888,1
450306,1110,12603,1
450307,12520,13280,1
450308,3340,6733,1


# Model recommendation

In [ ]:
# weights-001-0.4394.hdf5
model_path = '/content/drive/MyDrive/Recommendation/store/'
model = load_model(model_path +'weights-001-0.4394.hdf5')
model

# DNS sampling

## Tạo negative sample với mẫu được lấy theo xác suất 

In [ ]:
def generate_negative_samples(data_train, model, negative_size=1, gamma=2, path ='/content/drive/MyDrive/Recommendation/data_negative/',id_start = None):
    negative_samples = []
    users = 0
    track = users
    max_user_id = data_train['new_customer_code'].max()
    
    if id_start == None:
      grouped = data_train[data_train['purchase_dummy'] == 1].groupby('new_customer_code')['new_sku']
      all_items = set(data_train['new_sku'].unique())

    else:
      grouped = data_train[(data_train['purchase_dummy'] == 1) & (data_train['new_customer_code']>=id_start) ].groupby('new_customer_code')['new_sku']
      all_items = set(data_train['new_sku'].unique())
      users = id_start
      track = users

    for user_id, ground_truth in grouped:
        unobserved_items = list(all_items - set(ground_truth))

        user_id_array = np.array([user_id] * len(unobserved_items))
        unobserved_items_array = np.array(unobserved_items)

        predictions = model.predict([user_id_array, unobserved_items_array])

        probabilities = np.power(predictions, gamma)
        probabilities /= np.sum(probabilities)

        negative_samples.extend([(user_id, item_id, 0) for item_id in np.random.choice(unobserved_items, size=len(ground_truth)*negative_size, replace=False, p=probabilities.flatten())])
        users += 1
        print(users)

        # Nếu tới 4000 user thì sẽ track lại, ví dụ bắt đầu từ index 0 thì 4000 user có id từ 0 tới 3999
        # Nếu mà dùng user start thì sẽ bắt đầu tại id đó thay vì đi lại từ đầu
        # Sau đó làm trống negative_samples để tránh trùng data
        if ((users%4000 ==0) & (id_start == None)) | ((users%4000 ==0) & (users != id_start)):
            data = pd.DataFrame(negative_samples, columns=['new_customer_code', 'new_sku', 'purchase_dummy'])
            data.to_csv(path + 'negative_DNS_user_id_from_{}_to_{}.csv'.format(track,users-1),index= False)
            track = users
            print("đã track")
            negative_samples = []
        elif user_id == max_user_id:
            data = pd.DataFrame(negative_samples, columns=['new_customer_code', 'new_sku', 'purchase_dummy'])
            data.to_csv(path + 'negative_DNS_user_id_from_{}_to_{}.csv'.format(track,users-1),index= False)
            track = users
            print("đã track")
            negative_samples = []

In [ ]:
negative_samples = generate_negative_samples(data_train, model,id_start=22000)

In [ ]:
path ='/content/drive/MyDrive/Recommendation/data_negative/'

In [ ]:
data1 = pd.read_csv(path + 'negative_DNS_user_id_from_0_to_3999.csv' )
data2 = pd.read_csv(path + 'negative_DNS_user_id_from_4000_to_7999.csv' )
data3 = pd.read_csv(path + 'negative_DNS_user_id_from_8000_to_11999.csv' )
data4 = pd.read_csv(path + 'negative_DNS_user_id_from_12000_to_15999.csv' )
data5 = pd.read_csv(path + 'negative_DNS_user_id_from_16000_to_19999.csv' )
data6 = pd.read_csv(path + 'negative_DNS_user_id_from_20000_to_21999.csv' )
data7 = pd.read_csv(path + 'negative_DNS_user_id_from_22000_to_23999.csv' )
data8 = pd.read_csv(path + 'negative_DNS_user_id_from_24000_to_25999.csv' )
data9 = pd.read_csv(path + 'negative_DNS_user_id_from_26000_to_27698.csv' )

In [ ]:
df_combined = pd.concat([data_train,data1,data2,data3,data4,data5,data6,data7,data8,data9], ignore_index=True)
df_combined

,new_customer_code,new_sku,purchase_dummy
0,0,0,1
1,1,1,1
2,1,2,1
3,2,3,1
4,2,4,1
...,...,...,...
5580531,27698,1721,0
5580532,27698,3543,0
5580533,27698,5898,0
5580534,27698,5881,0


In [ ]:
df_combined.to_csv(path + 'train_negative_dns.csv', index = False)

## Tạo negative sample dựa trên một model recommendation đã được train trước đó

In [ ]:
def generate_negative_samples_fast(data_train, model, negative_size=1, gamma=2, path ='/content/drive/MyDrive/Recommendation/data_negative/',id_start = None):
    negative_samples = []
    users = 0
    track = users
    max_user_id = data_train['new_customer_code'].max()
    
    if id_start == None:
      grouped = data_train[data_train['purchase_dummy'] == 1].groupby('new_customer_code')['new_sku']
      all_items = set(data_train['new_sku'].unique())

    else:
      grouped = data_train[(data_train['purchase_dummy'] == 1) & (data_train['new_customer_code']>=id_start) ].groupby('new_customer_code')['new_sku']
      all_items = set(data_train['new_sku'].unique())
      users = id_start
      track = users

    for user_id, ground_truth in grouped:
        unobserved_items = list(all_items - set(ground_truth))
        choice_random =  [item_id for item_id in np.random.choice(unobserved_items, size=len(ground_truth)*4*negative_size, replace=False)]
        user_id_array = np.array([user_id] * len(choice_random))
        unobserved_items_array = np.array(choice_random)
        predictions = model.predict([user_id_array, unobserved_items_array])
        probabilities = np.power(predictions, gamma)
        probabilities /= np.sum(probabilities)
        negative_samples.extend([(user_id, item_id, 0) for item_id in np.random.choice(choice_random, 
                                                                                             size=len(ground_truth)*negative_size, 
                                                                                             replace=False, 
                                                                                             p=probabilities.flatten())])
        users += 1
        print(users)
        # Nếu tới 4000 user thì sẽ track lại, ví dụ bắt đầu từ index 0 thì 4000 user có id từ 0 tới 3999
        # Nếu mà dùng user start thì sẽ bắt đầu tại id đó thay vì đi lại từ đầu
        # Sau đó làm trống negative_samples để tránh trùng data
        if ((users%4000 ==0) & (id_start == None)) | ((users%4000 ==0) & (users != id_start)):
            data = pd.DataFrame(negative_samples, columns=['new_customer_code', 'new_sku', 'purchase_dummy'])
            data.to_csv(path + 'negative_DNS_user_id_from_{}_to_{}_fastx4.csv'.format(track,users-1),index= False)
            track = users
            print("đã track")
            negative_samples = []
        elif user_id == max_user_id:
            data = pd.DataFrame(negative_samples, columns=['new_customer_code', 'new_sku', 'purchase_dummy'])
            data.to_csv(path + 'negative_DNS_user_id_from_{}_to_{}_fastx4.csv'.format(track,users-1),index= False)
            track = users
            print("đã track")
            negative_samples = []

In [ ]:
negative_samples = generate_negative_samples_fast(data_train, model,id_start=None)

In [ ]:
data1 = pd.read_csv(path + 'negative_DNS_user_id_from_0_to_3999_fast.csv' )
data2 = pd.read_csv(path + 'negative_DNS_user_id_from_4000_to_7999_fast.csv' )
data3 = pd.read_csv(path + 'negative_DNS_user_id_from_8000_to_11999_fast.csv' )
data4 = pd.read_csv(path + 'negative_DNS_user_id_from_12000_to_15999_fast.csv' )
data5 = pd.read_csv(path + 'negative_DNS_user_id_from_16000_to_19999_fast.csv' )
data6 = pd.read_csv(path + 'negative_DNS_user_id_from_20000_to_23999_fast.csv' )
data7 = pd.read_csv(path + 'negative_DNS_user_id_from_24000_to_27698_fast.csv' )

In [ ]:
df_combined = pd.concat([data_train,data1,data2,data3,data4,data5,data6,data7], ignore_index=True)
df_combined

,new_customer_code,new_sku,purchase_dummy
0,0,0,1
1,1,1,1
2,1,2,1
3,2,3,1
4,2,4,1
...,...,...,...
5580531,27698,4487,0
5580532,27698,1818,0
5580533,27698,675,0
5580534,27698,5243,0


In [ ]:
df_combined.to_csv(path + 'train_negative_dns_fast.csv', index = False)